In [1]:

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from transformers import MarianMTModel, MarianTokenizer

class MultiTaskModel(nn.Module):
    def __init__(self):
        super(MultiTaskModel, self).__init__()
        self.digit_recognition = nn.Sequential(
            nn.Linear(784, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )
        self.house_price_prediction = nn.Sequential(
            nn.Linear(2, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
        self.language_translation_model = MarianMTModel.from_pretrained('Helsinki-NLP/opus-mt-en-fr')
        self.language_translation_tokenizer = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-en-fr')

    def forward(self, x, task):
        if task == 'digit_recognition':
            x = x.view(-1, 784)
            return self.digit_recognition(x)
        elif task == 'house_price_prediction':
            return self.house_price_prediction(x)
        elif task == 'language_translation':
            inputs = self.language_translation_tokenizer(x, return_tensors='pt')
            outputs = self.language_translation_model.generate(**inputs)
            translated_text = self.language_translation_tokenizer.decode(outputs[0], skip_special_tokens=True)
            return translated_text

model = MultiTaskModel()

# Train the digit recognition model
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
for epoch in range(10):
    for x, y in train_loader:
        optimizer.zero_grad()
        output = model(x, 'digit_recognition')
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

# Train the house price prediction model
data = torch.tensor([[1000, 2], [1200, 3], [1500, 4], [1800, 5], [2000, 6]], data=torch.float32)
labels = torch.tensor([200000, 250000, 300000, 350000, 400000], dtype=torch.float32).view(-1, 1)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
for epoch in range(100):
    optimizer.zero_grad()
    output = model(data, 'house_price_prediction')
    loss = criterion(output, labels)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")


KeyboardInterrupt: 